### Querying in Python
________________________________________
`from sqlalchemy import create_engine`

`import pandas as pd`

`engine = create_engine('....')`
_________________________________________
#### THIS:
`with engine as con:`

`rs = con.execute("SELECT * FROM table)`

`df = pd.DataFrame(rs.fetchmany(size=5))`

`df.columns = rs.keys()`

#### OR THIS:
`df = pd.read_sql_query("SELECT * FROM table", engine)`

`table_names = engine.table_names()`

`print(table_names)`


Database file: 'sqlite_db_pythonsqlite.db'

In [1]:
# import packages
from sqlalchemy import create_engine
import pandas as pd

import sqlite3 as sql

In [2]:
# create connection to database file
database_file = 'sqlite_db_pythonsqlite.db'
database = database_file
connection = sql.connect(database)

In [3]:
#test query
query = '''SELECT * FROM Bookings'''

In [4]:
#test dataframe
df = pd.read_sql_query(query, connection)
df.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


### Q10: 
Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [5]:
query10 = '''SELECT facid, name, SUM(cost) as totalcost
FROM (SELECT b.facid, f.name, 
CASE WHEN b.memid = 0 THEN f.guestcost
ELSE f.membercost END AS cost
FROM Bookings AS b
LEFT JOIN Facilities as f
ON f.facid = b.facid) as subquery
GROUP BY facid, name
HAVING totalcost < 1000
ORDER BY totalcost;'''
df10 = pd.read_sql_query(query10, connection)
df10.head()

,facid,name,totalcost
0,3,Table Tennis,90.0
1,7,Snooker Table,115.0
2,8,Pool Table,265.0
3,2,Badminton Court,604.5


### Q11: 
Produce a report of members and who recommended them in alphabetic surname,firstname order

In [6]:
query11 = '''SELECT (m.surname || ', ' || m.firstname) as membername, (m2.surname ||', ' || m2.firstname) AS recommendedby
FROM Members as m
INNER JOIN Members as m2
ON m.recommendedby = m2.memid
WHERE m.recommendedby > 0
ORDER by membername;'''
df11 = pd.read_sql_query(query11, connection)
df11.head()

,membername,recommendedby
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"


### Q12: 
Find the facilities with their usage by member, but not guests

In [7]:
query12 = '''SELECT b.facid, COUNT(b.memid) as memberbookings, f.name
FROM Bookings as b
LEFT JOIN Facilities as f
ON f.facid = b.facid
WHERE memid <> 0
GROUP BY b.facid;'''
df12 = pd.read_sql_query(query12, connection)
df12.head()

,facid,memberbookings,name
0,0,308,Tennis Court 1
1,1,276,Tennis Court 2
2,2,344,Badminton Court
3,3,385,Table Tennis
4,4,421,Massage Room 1


### Q13: 
Find the facilities usage by month, but not guests

In [24]:
query13 = '''SELECT f.name, 
CASE WHEN strftime('%m', starttime) = '07' THEN 'July' 
WHEN strftime('%m', starttime) = '08' THEN 'August' 
WHEN strftime('%m', starttime) = '09' THEN 'September' END AS month, 
COUNT(bookid) as monthlyuse
FROM Bookings as b
LEFT JOIN Facilities as f
ON b.facid = f.facid
WHERE memid <> 0
GROUP BY b.facid, month 
ORDER BY 3 DESC;'''
df13 = pd.read_sql_query(query13, connection)
df13.head(20)

,name,month,monthlyuse
0,Pool Table,September,408
1,Pool Table,August,272
2,Snooker Table,September,199
3,Table Tennis,September,194
4,Massage Room 1,September,191
5,Badminton Court,September,161
6,Snooker Table,August,154
7,Massage Room 1,August,153
8,Table Tennis,August,143
9,Tennis Court 1,September,132
